In [14]:
import pandas as pd

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Read the data
graph = pd.read_csv('best_results_graph.csv')
hypergraph = pd.read_csv('best_results_hypergraph.csv')
cell = pd.read_csv('best_results_cell.csv')
simplicial = pd.read_csv('best_results_simplicial.csv')

In [15]:
col_order = ['Model', 
'Cora',  'citeseer',  'PubMed', 
'MUTAG', 'PROTEINS', 'NCI1', 'NCI109','IMDB-BINARY', 'IMDB-MULTI', 'REDDIT-BINARY',
'US-county-demos-Election', 'US-county-demos-BachelorRate', 'US-county-demos-BirthRate', 'US-county-demos-DeathRate', 'US-county-demos-MedianIncome', 'US-county-demos-MigraRate', 'US-county-demos-UnemploymentRate',
'amazon_ratings', 'minesweeper', 'questions', 'roman_empire', 'tolokers',
'ZINC',]

rename_dict = {
    'Model': 'Model',
    'Cora': 'Cora',
    'citeseer': 'Citeseer',
    'PubMed': 'Pubmed',
    'MUTAG': 'MUTAG',
    'PROTEINS': 'PROTEINS',
    'NCI1': 'NCI1',
    'NCI109': 'NCI109',
    'IMDB-BINARY': 'IMDB-BINARY',
    'IMDB-MULTI': 'IMDB-MULTI',
    'REDDIT-BINARY': 'REDDIT-BINARY',
    
    'ZINC': 'ZINC', 
    'US-county-demos-Election': 'Election',
    'US-county-demos-BachelorRate': 'Bachelor Rate',
    'US-county-demos-BirthRate': 'Birth Rate',
    'US-county-demos-DeathRate': 'Death Rate',
    'US-county-demos-MedianIncome': 'Median Income',
    'US-county-demos-MigraRate': 'Migra Rate',
    'US-county-demos-UnemploymentRate': 'Unemployment Rate',
    'roman_empire': 'RomanEmpire',
    'amazon_ratings': 'Amazon Ratings',
    'minesweeper': 'Minesweeper',
    'questions': 'Questions',
    'tolokers': 'Tolokers'
}

rename_models = { 
    'gcn': 'GCN',
    'gat': 'GAT',
    'gin': 'GIN',

    'allsettransformer': 'AllSetTransformer',
    'edgnn': 'EDGNN',
    'unignn2': 'UniGNN2',

    'ccxn': 'CCXN',
    'cwn_dcm': 'CCCN',
    'cwn': 'CWN',

    'scn': 'SCN',
    'sccn': 'SCCN',
    'sccnn_custom': 'SCCNN',
}

df = pd.concat([graph, hypergraph, cell, simplicial], axis=0)
df = df[col_order]
df = df.rename(columns=rename_dict)
df['Model'] = df['Model'].map(rename_models)
df['Model'].reset_index(drop=True, inplace=True)

# Give correct order of models
df.set_index('Model', inplace=True)
model_order = ['GCN', 'GAT', 'GIN', 'AllSetTransformer', 'EDGNN', 'UniGNN2', 'CWN', 'CCCN','CCXN', 'SCN', 'SCCN', 'SCCNN']
df = df.T[model_order].T

# substitute off nan with NA
df.fillna('NA', inplace=True)

# fill nan ± nan with NA
for col in df.columns:
    df[col] = df[col].apply(lambda x: 'NA' if x == 'nan ± nan' else x)

In [17]:
df.to_csv('main_results.csv')

In [4]:
import re
import numpy as np

def get_metric(s, metric=0):
    '''
    0 - mean
    1 - std
    '''
    if type(s) == str:
        vals = [v.strip() for v in s.split('±')]
        if len(vals) == 1:
            return -1
        else:
            return float(vals[metric])
    else:
        return float('nan')

def highlight_best(row):
    if row.name == 'avg. ranking':
        row = [float(v) for v in row.values]
        return np.where(row == np.nanmin(row), f'background: grey; font-weight: bold;', None)
    elif get_metric(row[1],0 ) == -1:
        # First columns
        return [None] * len(row)
    elif row.name in ['Election','Bachelor Rate', 'Birth Rate','Death Rate', 'Median Income', 'Migra Rate','Unemployment Rate','ZINC']:
        x_mean = np.array([get_metric(val, 0) for val in row])
        x_std = np.array([get_metric(val, 1) for val in row])
        best_mean = np.nanmin(x_mean)
        best_idx = np.nanargmin(x_mean)
        best_std = x_std[np.nanargmin(x_mean)]
        blue_results = x_mean < (best_mean + best_std)
        blue_results[best_idx] = False
        grey_results = x_mean == best_mean

        styles = np.array([None] * len(blue_results))
        styles[blue_results] = 'background: blue;'
        styles[grey_results] = 'background: grey; font-weight: bold;'

        return styles
    else:
        x_mean = np.array([get_metric(val, 0) for val in row])
        x_std = np.array([get_metric(val, 1) for val in row])
        best_mean = np.nanmax(x_mean)
        best_idx = np.nanargmax(x_mean)
        best_std = x_std[np.nanargmax(x_mean)]
        blue_results = x_mean > (best_mean - best_std)
        blue_results[best_idx] = False
        grey_results = x_mean == best_mean

        styles = np.array([None] * len(blue_results))
        styles[blue_results] = 'background: blue;'
        styles[grey_results] = 'background: grey; font-weight: bold;'

        return styles
        #return np.where(x_mean == np.nanmax(x_mean), f'background: grey; font-weight: bold;', None)

def replace_style_incl(text, name, new_name):
    return re.sub(r'[\\]' + name + '\s([^&]+?)\s([\\\\&])',
                  '\\\\' + new_name + '{\g<1>} \g<2>', text)

def replace_style_normal(text, name, new_name):
    return text.replace(name, new_name)

def beautify_table(df):
    # Highlight the cell with a blue background
    highlighted_df = df.style.apply(highlight_best, axis=0)
    highlighted_df = highlighted_df.hide(axis="index")
    return highlighted_df


def convert_to_latex(pivot_df):
    # Highlight cells
    highlighted_df = beautify_table(pivot_df)

    # Convert the table to LaTeX format
    latex_table = highlighted_df.to_latex(column_format='', hrules=True)
    # Remove \begin and \end and other
    latex_table = latex_table.replace('\\begin{tabular}{}', '')
    latex_table = latex_table.replace('\\end{tabular}', '')
    latex_table = latex_table.replace('\\toprule', '')
    latex_table = latex_table.replace('\\bottomrule', '')
    latex_table = latex_table.replace('\\midrule', '\\hline')
    #latex_table = latex_table.replace('\\\\', '\\\\ [5pt]')

    latex_table = replace_style_incl(latex_table, 'font-weightbold', 'textbf')
    latex_table = replace_style_normal(latex_table, '\\backgroundblue', '\\cellcolor{blue!25}')
    latex_table = replace_style_normal(latex_table, '\\backgroundgrey', '\\cellcolor{grey!25}')

    latex_table = f'''
        \\hline
        {latex_table}
        '''

    return latex_table + '\n'


In [5]:
df.reset_index(inplace=True)

In [6]:
df

,Model,Cora,Citeseer,Pubmed,MUTAG,PROTEINS,NCI1,NCI109,IMDB-BINARY,IMDB-MULTI,REDDIT-BINARY,Election,Bachelor Rate,Birth Rate,Death Rate,Median Income,Migra Rate,Unemployment Rate,Amazon Ratings,Minesweeper,Questions,RomanEmpire,Tolokers,ZINC
0,GCN,87.09 ± 0.2,75.53 ± 1.27,89.4 ± 0.3,69.79 ± 6.8,75.7 ± 2.14,72.86 ± 0.69,72.2 ± 1.22,72.0 ± 2.48,49.97 ± 2.16,76.24 ± 0.54,0.3062 ± 0.0215,0.2928 ± 0.022,0.7171 ± 0.0903,0.5081 ± 0.0389,0.2218 ± 0.0256,0.7951 ± 0.1184,0.2531 ± 0.0312,49.56 ± 0.55,87.52 ± 0.42,76.11 ± 0.82,78.16 ± 0.32,83.02 ± 0.71,0.6217 ± 0.0088
1,GAT,86.71 ± 0.95,74.41 ± 1.75,89.44 ± 0.24,72.77 ± 2.77,76.34 ± 1.66,75.0 ± 0.99,73.8 ± 0.73,69.76 ± 2.65,50.13 ± 3.87,75.68 ± 1.0,0.2908 ± 0.0227,0.2842 ± 0.0221,0.7148 ± 0.0939,0.5108 ± 0.0369,0.2038 ± 0.0219,0.7722 ± 0.1279,0.2288 ± 0.0278,50.17 ± 0.59,89.64 ± 0.43,77.89 ± 0.72,84.02 ± 0.51,84.43 ± 1.0,0.6059 ± 0.0058
2,GIN,87.21 ± 1.89,73.73 ± 1.23,89.29 ± 0.41,79.57 ± 6.13,75.2 ± 3.3,74.26 ± 0.96,74.42 ± 0.7,70.96 ± 1.93,47.68 ± 4.21,81.96 ± 1.36,0.2841 ± 0.0151,0.3073 ± 0.0287,0.718 ± 0.0876,0.5231 ± 0.0444,0.2137 ± 0.0238,0.797 ± 0.1036,0.2212 ± 0.0239,49.16 ± 1.02,87.82 ± 0.34,76.38 ± 0.88,79.56 ± 0.2,80.72 ± 1.19,0.5713 ± 0.0371
3,AllSetTransformer,88.92 ± 0.44,73.85 ± 2.21,89.62 ± 0.25,71.06 ± 6.49,76.63 ± 1.74,75.18 ± 1.24,73.75 ± 1.09,70.32 ± 3.27,50.51 ± 2.92,74.84 ± 2.68,0.2897 ± 0.0136,0.2996 ± 0.0276,0.7111 ± 0.0823,0.4938 ± 0.0456,0.2066 ± 0.0216,0.7775 ± 0.123,0.2199 ± 0.022,50.5 ± 0.27,81.14 ± 0.05,NA,79.5 ± 0.13,83.26 ± 0.1,0.59 ± 0.0233
4,EDGNN,87.06 ± 1.09,74.93 ± 1.39,89.04 ± 0.51,80.0 ± 4.9,73.91 ± 4.39,73.97 ± 0.82,74.93 ± 2.5,69.12 ± 2.92,49.17 ± 4.35,83.24 ± 1.45,0.3418 ± 0.0246,0.2934 ± 0.0242,0.7036 ± 0.0743,0.5192 ± 0.0466,0.225 ± 0.0236,0.7988 ± 0.1225,0.2576 ± 0.0281,48.18 ± 0.09,84.52 ± 0.05,NA,81.01 ± 0.24,77.53 ± 0.01,0.5109 ± 0.011
5,UniGNN2,86.97 ± 0.88,74.72 ± 1.08,89.34 ± 0.45,80.43 ± 4.09,75.2 ± 2.96,73.02 ± 0.92,70.76 ± 1.11,71.04 ± 1.31,49.76 ± 3.55,75.56 ± 3.19,0.367 ± 0.0204,0.311 ± 0.0229,0.7257 ± 0.0952,0.511 ± 0.0454,0.2342 ± 0.0218,0.7923 ± 0.1162,0.2833 ± 0.0207,49.06 ± 0.08,78.02 ± 0.0,NA,77.06 ± 0.2,77.35 ± 0.03,0.5991 ± 0.0056
6,CWN,86.32 ± 1.38,75.2 ± 1.82,88.64 ± 0.36,80.43 ± 1.78,76.13 ± 2.7,73.93 ± 1.87,73.8 ± 2.06,70.4 ± 2.02,49.71 ± 2.83,85.52 ± 1.38,0.3437 ± 0.0216,0.3306 ± 0.0279,0.7181 ± 0.086,0.5399 ± 0.0553,0.2468 ± 0.03,0.838 ± 0.1286,0.2535 ± 0.031,51.9 ± 0.15,88.62 ± 0.04,NA,81.81 ± 0.62,NA,0.3444 ± 0.0118
7,CCCN,87.44 ± 1.28,75.63 ± 1.58,88.52 ± 0.44,77.02 ± 9.32,73.33 ± 2.3,76.67 ± 1.48,75.35 ± 1.5,69.12 ± 2.82,47.79 ± 3.45,85.12 ± 1.29,0.307 ± 0.0155,0.313 ± 0.0248,0.7134 ± 0.0915,0.5443 ± 0.0568,0.2263 ± 0.0187,0.8373 ± 0.1206,0.2446 ± 0.0317,50.26 ± 0.17,89.42 ± 0.0,NA,82.14 ± 0.0,NA,0.3358 ± 0.0157
8,CCXN,86.79 ± 1.81,74.67 ± 2.24,88.91 ± 0.47,74.89 ± 5.51,75.63 ± 2.57,74.86 ± 0.82,75.66 ± 1.3,70.08 ± 1.21,47.63 ± 3.45,82.84 ± 2.54,0.3471 ± 0.0154,0.3166 ± 0.0288,0.745 ± 0.1123,0.5426 ± 0.055,0.2489 ± 0.0296,0.8482 ± 0.185,0.2707 ± 0.0258,48.93 ± 0.14,88.88 ± 0.36,NA,81.44 ± 0.31,NA,0.3977 ± 0.0391
9,SCN,82.27 ± 1.34,71.24 ± 1.68,88.72 ± 0.5,73.62 ± 6.13,75.27 ± 2.14,74.49 ± 1.03,75.7 ± 1.04,67.52 ± 4.5,49.87 ± 1.64,71.28 ± 2.06,0.4648 ± 0.043,0.3186 ± 0.0241,0.7122 ± 0.0836,0.5208 ± 0.0525,0.2521 ± 0.0253,0.9209 ± 0.1993,0.3753 ± 0.0432,NA,90.32 ± 0.11,NA,88.79 ± 0.46,NA,0.5315 ± 0.0401


In [7]:
list(df.columns)
column1 = ['Model',
 'Cora',
 'Citeseer',
 'Pubmed',
 'MUTAG',
 'PROTEINS',
 'NCI1',
 'NCI109',
 'IMDB-BINARY',
 'IMDB-MULTI',
 'REDDIT-BINARY',
 'Election',]

column2 = ['Model',
 'Bachelor Rate',
 'Birth Rate',
 'Death Rate',
 'Median Income',
 'Migra Rate',
 'Unemployment Rate',
 'Amazon Ratings',
 'Minesweeper',
 
 'RomanEmpire',
 'Tolokers',
 'ZINC']


latex_table = ''

latex_table += convert_to_latex(df[column1])
latex_table += convert_to_latex(df[column2]) 
latex_table += '\\midrule'



latex_table = latex_table.replace('\hline', '\midrule')
latex_table = latex_table.replace('grey', 'gray')
print(latex_table)


        \midrule
        

Model & Cora & Citeseer & Pubmed & MUTAG & PROTEINS & NCI1 & NCI109 & IMDB-BINARY & IMDB-MULTI & REDDIT-BINARY & Election \\
\midrule
GCN & 87.09 ± 0.2 & \cellcolor{blue!25} 75.53 ± 1.27 & \cellcolor{blue!25} 89.4 ± 0.3 & 69.79 ± 6.8 & \cellcolor{blue!25} 75.7 ± 2.14 & 72.86 ± 0.69 & 72.2 ± 1.22 & \cellcolor{gray!25} \textbf{72.0 ± 2.48} & \cellcolor{blue!25} 49.97 ± 2.16 & 76.24 ± 0.54 & 0.3062 ± 0.0215 \\
GAT & 86.71 ± 0.95 & \cellcolor{blue!25} 74.41 ± 1.75 & \cellcolor{blue!25} 89.44 ± 0.24 & 72.77 ± 2.77 & \cellcolor{blue!25} 76.34 ± 1.66 & 75.0 ± 0.99 & 73.8 ± 0.73 & \cellcolor{blue!25} 69.76 ± 2.65 & \cellcolor{blue!25} 50.13 ± 3.87 & 75.68 ± 1.0 & \cellcolor{blue!25} 0.2908 ± 0.0227 \\
GIN & 87.21 ± 1.89 & 73.73 ± 1.23 & 89.29 ± 0.41 & \cellcolor{blue!25} 79.57 ± 6.13 & \cellcolor{blue!25} 75.2 ± 3.3 & 74.26 ± 0.96 & 74.42 ± 0.7 & \cellcolor{blue!25} 70.96 ± 1.93 & \cellcolor{blue!25} 47.68 ± 4.21 & 81.96 ± 1.36 & \cellcolor{gray!25} \textbf{0.2841 ±

# Ablation table

In [18]:
cell = pd.read_csv('ablation_cellular.csv')
simplicial = pd.read_csv('ablation_simplicial.csv')


In [19]:
df = pd.concat([cell, simplicial], axis=0, )
df.drop('Unnamed: 0', axis=1, inplace=True)
df = df.rename(columns=rename_dict)
df['Model'] = df['Model'].apply(lambda x:rename_models[x.strip(' ')])


In [10]:
df['Model']

0     CCXN
1     CCXN
2      CWN
3      CWN
4     CCCN
5     CCCN
0     SCCN
1     SCCN
2    SCCNN
3    SCCNN
4      SCN
5      SCN
Name: Model, dtype: object

In [11]:
# make multiindex from Model and ReadOut

df.set_index(['Model', 'ReadOut'], inplace=True)


In [12]:
column1 = [
 'Cora',
 'Citeseer',
 'Pubmed',
 'MUTAG',
 'PROTEINS',
 'NCI1',
 'NCI109',
 'IMDB-BINARY',
 'IMDB-MULTI',
 'REDDIT-BINARY',
 ]

column2 = [
 'Election',
 'Bachelor Rate',
 'Birth Rate',
 'Death Rate',
 'Median Income',
 'Migra Rate',
 'Unemployment Rate',
 'Minesweeper',
 'RomanEmpire',
 'ZINC']


latex_table = ''
latex_table += df[column1].to_latex()
latex_table += df[column2].to_latex()
print(latex_table)

\begin{tabular}{llllllllllll}
\toprule
 &  & Cora & Citeseer & Pubmed & MUTAG & PROTEINS & NCI1 & NCI109 & IMDB-BINARY & IMDB-MULTI & REDDIT-BINARY \\
Model & ReadOut &  &  &  &  &  &  &  &  &  &  \\
\midrule
\multirow[t]{2}{*}{CCXN} & NoReadOut & 86.32 ± 1.22 & 72.87 ± 1.13 & 88.91 ± 0.47 & 69.79 ± 4.61 & 75.63 ± 2.57 & 72.43 ± 1.72 & 73.22 ± 0.48 & 70.08 ± 1.21 & 47.63 ± 3.45 & 74.4 ± 1.5 \\
 & PropagateSignalDown & 86.79 ± 1.81 & 74.67 ± 2.24 & 88.38 ± 0.38 & 74.89 ± 5.51 & 74.91 ± 1.85 & 74.86 ± 0.82 & 75.66 ± 1.3 & 68.96 ± 2.03 & 48.75 ± 3.56 & 82.84 ± 2.54 \\
\cline{1-12}
\multirow[t]{2}{*}{CWN} & NoReadOut & 74.95 ± 0.98 & 70.49 ± 2.85 & 86.94 ± 0.68 & 69.68 ± 8.58 & 76.13 ± 1.8 & 68.52 ± 0.51 & 68.19 ± 0.65 & 70.4 ± 2.02 & 49.71 ± 2.83 & 76.2 ± 0.86 \\
 & PropagateSignalDown & 86.32 ± 1.38 & 75.2 ± 1.82 & 88.64 ± 0.36 & 80.43 ± 1.78 & 76.13 ± 2.7 & 73.93 ± 1.87 & 73.8 ± 2.06 & 69.28 ± 2.57 & 49.87 ± 2.33 & 85.52 ± 1.38 \\
\cline{1-12}
\multirow[t]{2}{*}{CCCN} & NoReadOut & 87.4

In [13]:
df

Cora   IMDB-BINARY    IMDB-MULTI         MUTAG          NCI1        NCI109      PROTEINS        Pubmed REDDIT-BINARY    Bachelor Rate       Birth Rate       Death Rate         Election    Median Income       Migra Rate Unemployment Rate             ZINC Amazon Ratings      Citeseer   Minesweeper   RomanEmpire
Model ReadOut                                                                                                                                                                                                                                                                                                                                            
CCXN  NoReadOut            86.32 ± 1.22  70.08 ± 1.21  47.63 ± 3.45  69.79 ± 4.61  72.43 ± 1.72  73.22 ± 0.48  75.63 ± 2.57  88.91 ± 0.47    74.4 ± 1.5  0.3318 ± 0.0282  0.7996 ± 0.1227  0.5663 ± 0.0588  0.3898 ± 0.0451  0.2538 ± 0.0276  0.8046 ± 0.1133   0.2836 ± 0.0478  0.6282 ± 0.0181      nan ± nan  72.87 ± 1.13  88.88 ± 0.36  80.46 ± 0.23
      PropagateSignalDown  86.79 ± 1.81  68.96 ± 2.03  48.75 ± 3.56  74.89 ± 5.51  74.86 ± 0.82   75.66 ± 1.3  74.91 ± 1.85  88.38 ± 0.38  82.84 ± 2.54  0.3166 ± 0.0288   0.745 ± 0.1123   0.5426 ± 0.055  0.3471 ± 0.0154  0.2489 ± 0.0296   0.8482 ± 0.185   0.2707 ± 0.0258  0.3977 ± 0.0391      nan ± nan  74.67 ± 2.24  89.76 ± 0.32  81.44 ± 0.31
CWN   NoReadOut            74.95 ± 0.98   70.4 ± 2.02  49.71 ± 2.83  69.68 ± 8.58  68.52 ± 0.51  68.19 ± 0.65   76.13 ± 1.8  86.94 ± 0.68   76.2 ± 0.86  0.3306 ± 0.0279  0.8088 ± 0.1141  0.5532 ± 0.0494  0.5976 ± 0.0428  0.3631 ± 0.0373  0.8998 ± 0.1641   0.4628 ± 0.0362  0.6951 ± 0.0036   45.58 ± 0.25  70.49 ± 2.85   48.82 ± 0.0  66.13 ± 0.03
      PropagateSignalDown  86.32 ± 1.38  69.28 ± 2.57  49.87 ± 2.33  80.43 ± 1.78  73.93 ± 1.87   73.8 ± 2.06   76.13 ± 2.7  88.64 ± 0.36  85.52 ± 1.38  0.3331 ± 0.0319   0.7181 ± 0.086  0.5399 ± 0.0553  0.3437 ± 0.0216    0.2468 ± 0.03   0.838 ± 0.1286    0.2535 ± 0.031  0.3444 ± 0.0118    51.9 ± 0.15   75.2 ± 1.82  88.62 ± 0.04  81.81 ± 0.62
CCCN  NoReadOut            87.44 ± 1.28  69.12 ± 2.82  49.01 ± 2.63  80.85 ± 5.42  76.67 ± 1.48   75.35 ± 1.5  73.55 ± 3.43  88.52 ± 0.44  85.12 ± 1.29  0.3158 ± 0.0247  0.7134 ± 0.0915  0.5443 ± 0.0568   0.307 ± 0.0155  0.2263 ± 0.0187  0.8404 ± 0.1031   0.2446 ± 0.0317  0.3457 ± 0.0225   50.55 ± 0.15  75.63 ± 1.58   89.42 ± 0.0   82.14 ± 0.0
      PropagateSignalDown  87.68 ± 1.17  69.44 ± 2.46  47.79 ± 3.45  77.02 ± 9.32  77.65 ± 1.28  74.83 ± 1.18   73.33 ± 2.3  88.67 ± 0.39  83.32 ± 0.73   0.313 ± 0.0248  0.7183 ± 0.0479  0.5408 ± 0.0561  0.3058 ± 0.0057   0.228 ± 0.0189  0.8373 ± 0.1206   0.2499 ± 0.0252  0.3358 ± 0.0157   50.26 ± 0.17  74.91 ± 1.25   89.85 ± 0.0   82.51 ± 0.0
SCCN  NoReadOut            80.86 ± 2.16  69.68 ± 2.42  46.45 ± 1.78   70.64 ± 5.9  76.42 ± 0.88  75.49 ± 1.39  75.05 ± 2.76  88.04 ± 0.51  73.52 ± 1.53  0.3588 ± 0.0246  0.8242 ± 0.0942  0.5751 ± 0.0553  0.5344 ± 0.0323  0.2868 ± 0.0276  0.9146 ± 0.1822    0.4328 ± 0.044  0.5538 ± 0.0111            NaN   69.6 ± 1.83   88.85 ± 0.0   88.2 ± 0.22
      PropagateSignalDown  80.06 ± 1.66   69.3 ± 6.81  47.31 ± 3.12  73.62 ± 4.41  76.17 ± 1.39  75.31 ± 1.36  74.34 ± 3.17  88.37 ± 0.48  74.44 ± 1.74   0.341 ± 0.0249  0.8264 ± 0.1018  0.5629 ± 0.0444  0.5686 ± 0.0247   0.2908 ± 0.032   0.9303 ± 0.172   0.4734 ± 0.0377  0.4605 ± 0.0826            NaN   68.86 ± 2.4  89.07 ± 0.25  88.27 ± 0.14
SCCNN NoReadOut            82.19 ± 1.07  70.88 ± 2.25  48.75 ± 3.98  76.17 ± 6.63   76.6 ± 1.75  77.12 ± 1.07  74.19 ± 2.86  88.18 ± 0.32    73.0 ± 0.2   0.3394 ± 0.028  0.7937 ± 0.1162  0.5527 ± 0.0474  0.5112 ± 0.0316  0.2825 ± 0.0279  0.8976 ± 0.1431   0.4278 ± 0.0394   0.3562 ± 0.013            NaN  70.23 ± 2.69    87.4 ± 0.0  89.15 ± 0.32
      PropagateSignalDown  80.65 ± 2.39  69.28 ± 5.69  46.67 ± 3.13  70.64 ± 3.16   75.6 ± 2.45  75.43 ± 1.94  74.98 ± 1.92  87.78 ± 0.58   73.5 ± 0.42  0.3449 ± 0.0314  0.8251 ± 0.1184   0.579 ± 0.0541    0.557 ± 0.041  0.3073 ±